In [1]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import StandardScaler
from scipy import stats
from sklearn.linear_model import LinearRegression

In [2]:
data = pd.read_csv("datasets/features_with_politeness.csv", lineterminator='\n', encoding="ISO-8859-1").dropna()
print(len(data))
data.columns

10162


Index(['question_id', 'answer_id', 'a_score_rel_q_score', 'answer_score',
       'entities_matches', 'reputation', 'reply_by_author', 'len_answer_text',
       'is_accepted', 'code_snippet_count', 'link_count', 'Hedges',
       'Positive.Emotion', 'Negative.Emotion', 'Impersonal.Pronoun',
       'Swearing', 'Negation', 'Filler.Pause', 'Informal.Title',
       'Formal.Title', 'Could.You', 'Can.You', 'By.The.Way', 'Let.Me.Know',
       'Goodbye', 'For.Me', 'For.You', 'Reasoning', 'Reassurance',
       'Ask.Agency', 'Give.Agency', 'Hello', 'Please', 'First.Person.Plural',
       'First.Person.Single', 'Second.Person', 'Agreement', 'Acknowledgement',
       'Subjectivity', 'Bare.Command', 'WH.Questions', 'YesNo.Questions',
       'Gratitude', 'Apology', 'Truth.Intensifier', 'Affirmation',
       'Adverb.Just', 'Conjunction.Start'],
      dtype='object')

In [112]:
b_data = data.copy()
scaler = StandardScaler()
cols = (data.columns.tolist())
del cols[0:2]
cols.remove('answer_score')
b_data[cols] = scaler.fit_transform(b_data[cols])
b_data.sort_values(by=['question_id','answer_score'], ascending=False, inplace=True)
#b_data["rank"] = b_data.groupby("question_id")["answer_score"].rank(ascending=False)
print(max(b_data['answer_score']))
b_data

7467


question_id  answer_id  a_score_rel_q_score  answer_score  \
1574     67239221   67239259             0.175378             1   
1547     67237155   67237241             0.179703             0   
1343     67237061   67237260             0.175378             0   
1150     67236923   67236978             0.184029             5   
1149     67236923   67237336             0.173215             0   
...           ...        ...                  ...           ...   
783           826        834             0.123471            -4   
774           810        829             0.145099             5   
775           810        820             0.145099             5   
772           810        818             0.136448             1   
773           810        870             0.136448             1   

      entities_matches  reputation  reply_by_author  len_answer_text  \
1574         -0.425816   -0.421044        -0.185487        -0.495355   
1547         -0.425816   -0.308212        -0.185487        -0.130865   
1343         -0.425816    1.079331        -0.185487         1.586562   
1150          4.109973    0.011456        -0.185487        -0.841312   
1149          0.708131   -0.419148        -0.185487        -0.866023   
...                ...         ...              ...              ...   
783          -0.425816   -0.343975        -0.185487        -0.104095   
774          -0.425816    0.260550        -0.185487         1.304442   
775          -0.425816    0.583781        -0.185487         0.464262   
772          -0.425816   -0.160983         5.391205        -0.159695   
773           0.708131   -0.160983         5.391205         0.066824   

      is_accepted  code_snippet_count  ...  Bare.Command  WH.Questions  \
1574     1.837498            3.771277  ...      2.417250     -0.194408   
1547     1.837498            1.617715  ...     -0.414020     -0.194408   
1343     1.837498           -0.535847  ...      1.001615     -0.194408   
1150     1.837498            0.540934  ...     -0.414020     -0.194408   
1149    -0.544218            0.540934  ...     -0.414020     -0.194408   
...           ...                 ...  ...           ...           ...   
783     -0.544218           -0.535847  ...     -0.414020     -0.194408   
774     -0.544218           -0.535847  ...     -0.414020     -0.194408   
775      1.837498            0.540934  ...     -0.414020     -0.194408   
772     -0.544218           -0.535847  ...     -0.414020     -0.194408   
773     -0.544218           -0.535847  ...     -0.414020     -0.194408   

      YesNo.Questions  Gratitude   Apology  Truth.Intensifier  Affirmation  \
1574        -0.276662  -0.114404 -0.083912          -0.325376    -0.169873   
1547        -0.276662  -0.114404 -0.083912          -0.325376    -0.169873   
1343        -0.276662  -0.114404 -0.083912          -0.325376    -0.169873   
1150        -0.276662  -0.114404 -0.083912          -0.325376    -0.169873   
1149        -0.276662  -0.114404 -0.083912          -0.325376    -0.169873   
...               ...        ...       ...                ...          ...   
783         -0.276662  -0.114404 -0.083912          -0.325376    -0.169873   
774          1.337252  -0.114404 -0.083912          -0.325376    -0.169873   
775          1.337252  -0.114404 -0.083912           1.706875    -0.169873   
772         -0.276662  -0.114404 -0.083912          -0.325376    -0.169873   
773         -0.276662  -0.114404 -0.083912          -0.325376    -0.169873   

      Adverb.Just  Conjunction.Start  rank  
1574    -0.396733          -0.407037   1.0  
1547     1.412784          -0.407037   1.0  
1343     3.222301           2.646720   1.0  
1150    -0.396733          -0.407037   1.0  
1149    -0.396733           1.119841   2.0  
...           ...                ...   ...  
783     -0.396733          -0.407037   7.0  
774      3.222301           1.119841   1.5  
775      1.412784          -0.407037   1.5  
772      1.412784          -0.407037   3.5  
773      1.4127

In [113]:

m_data = b_data.drop(['answer_id', 'is_accepted'], axis=1)
test_set = m_data.iloc[0:2096].copy()

test_label = test_set['answer_score']


test = test_set.drop(['question_id', 'answer_score'], axis=1).to_numpy()
train_set = m_data.iloc[m_data.index[2096:]]
train_label = train_set['answer_score']
train = train_set.drop(['question_id', 'answer_score'], axis=1).to_numpy()
reg = LinearRegression().fit(train, train_label)
predictions = reg.predict(test)
print(predictions)
r = permutation_importance(clf, test, test_label,n_repeats=30,random_state=0)
r

[63.5485908  45.5656156  11.28121403 ... 46.50649973 -4.35833012
 10.30542682]


{'importances_mean': array([ 1.46310433e-03,  1.55852417e-03, -6.20229008e-04, -6.36132316e-05,
         4.05534351e-03,  2.01972010e-03,  3.27608142e-03,  3.95992366e-03,
         3.33969466e-03,  6.04325700e-04,  7.79262087e-04, -4.93002545e-04,
         3.92811705e-03,  4.77099237e-05,  1.60623410e-03,  4.23027990e-03,
         9.70101781e-04,  1.06552163e-03,  1.59033079e-05, -3.49872774e-04,
        -1.59033079e-05,  1.90839695e-04,  6.20229008e-04, -1.59033079e-05,
        -3.18066158e-05, -6.20229008e-04, -4.24618321e-03,  3.33969466e-04,
        -6.99745547e-04,  8.42875318e-04, -7.95165394e-05,  1.87659033e-03,
         5.24809160e-04,  1.59033079e-05,  2.17875318e-03,  1.49491094e-03,
         3.18066158e-05, -2.33778626e-03, -3.18066158e-05, -3.97582697e-04,
         9.86005089e-04,  1.74936387e-04,  1.49491094e-03,  6.36132316e-05]),
 'importances_std': array([2.54532430e-03, 3.11623685e-03, 2.09108845e-03, 1.35915144e-03,
        3.32089858e-03, 2.61516347e-03, 1.77974998e

In [106]:
#b_data['is_accepted'] = b_data['is_accepted'].astype(int).to_numpy()
#print(np.bincount(target))
# Drop non-discrete columns and convert columns


to_binary_cols = ['Hedges', 'Positive.Emotion', 'Negative.Emotion',
       'Impersonal.Pronoun', 'Swearing', 'Negation', 'Filler.Pause',
       'Informal.Title', 'Formal.Title', 'Could.You', 'Can.You', 'By.The.Way',
       'Let.Me.Know', 'Goodbye', 'For.Me', 'For.You', 'Reasoning',
       'Reassurance', 'Ask.Agency', 'Give.Agency', 'Hello', 'Please',
       'First.Person.Plural', 'First.Person.Single', 'Second.Person',
       'Agreement', 'Acknowledgement', 'Subjectivity', 'Bare.Command',
       'WH.Questions', 'YesNo.Questions', 'Gratitude', 'Apology',
       'Truth.Intensifier', 'Affirmation', 'Adverb.Just', 'Conjunction.Start', 
        'code_snippet_count', 'link_count', 'entities_matches']

for col in to_binary_cols:
    b_data[col] = b_data[col].apply(lambda x: 0 if x == 0 else 1)
    
avg_len = b_data['len_answer_text'].mean()
avg_rep = b_data['reputation'].mean()

b_data['len_answer_text'] = b_data['len_answer_text'].apply(lambda x: 0 if x < avg_len else 1)
b_data['reputation'] = b_data['reputation'].apply(lambda x: 0 if x < avg_rep else 1)
b_data['has_more_upvotes_than_ans'] = b_data['a_score_rel_q_score'].apply(lambda x: 0 if x < 0 else 1)
b_data['reply_by_author'] = b_data["reply_by_author"].astype(int)

bins = [-20, 0, 2, 5, 7500]
target,width_bins = pd.cut(b_data['answer_score'], bins, retbins=True, duplicates='drop')

width_binned_counts = target.value_counts()

b_data['answer_score'] = target.cat.codes


m_data = b_data.drop(['answer_id', 'a_score_rel_q_score', 'is_accepted'], axis=1)

test_set = m_data.iloc[0:2096].copy()

test_label = test_set['answer_score']


test = test_set.drop(['question_id', 'rank','answer_score'], axis=1).to_numpy()
train_set = m_data.iloc[m_data.index[2096:]]
train_label = train_set['answer_score']
train = train_set.drop(['question_id', 'rank','answer_score'], axis=1).to_numpy()
print(test)

clf = BernoulliNB()
clf.fit(train, train_label)
predictions = clf.predict(test)
print(predictions)

test_set['pred'] = predictions
rankings = test_set.groupby("question_id")["pred"].rank()
print(rankings)

print(np.bincount(predictions))


r = permutation_importance(clf, test, test_label,n_repeats=30,random_state=0)


stats.spearmanr(rankings, test_set['rank'])



[[0 0 0 ... 0 0 1]
 [0 0 0 ... 1 0 1]
 [0 1 0 ... 1 1 1]
 ...
 [1 0 0 ... 0 0 1]
 [0 1 0 ... 0 0 1]
 [1 0 0 ... 0 0 1]]
[2 3 3 ... 1 3 2]
1574    1.0
1547    1.0
1343    1.0
1150    2.0
1149    1.0
       ... 
8392    6.5
8386    2.0
8373    1.0
8374    3.0
8375    2.0
Name: pred, Length: 2096, dtype: float64
[771 779  86 460]


SpearmanrResult(correlation=0.39373488851050986, pvalue=1.1205504605537895e-78)

In [104]:
clf = LogisticRegression(multi_class='multinomial',solver ='newton-cg').fit(train,train_label)
predictions = clf.predict(test)
print(predictions)
print(np.bincount(predictions))

test_set['pred'] = predictions
rankings = test_set.groupby("question_id")["pred"].rank(ascending=False)

r = permutation_importance(clf, test, test_label,n_repeats=30,random_state=0)
print(r.importances_mean)

stats.spearmanr(rankings, test_set['rank'])


[3 3 3 ... 1 3 2]
[663 904  84 445]
[ 3.49872774e-03  4.26208651e-03  2.56043257e-03  2.13740458e-02
  1.40585242e-02  6.85432570e-03  3.30788804e-03 -1.31997455e-03
 -1.25636132e-03 -2.54452926e-04 -1.00190840e-03  6.77480916e-03
  5.24809160e-04  1.49491094e-03 -4.13486005e-04 -5.72519084e-04
  8.90585242e-04 -1.27226463e-04  3.18066158e-05 -7.40148683e-18
 -9.54198473e-04 -7.63358779e-04 -7.40148683e-18  1.00190840e-03
  7.63358779e-04 -1.60623410e-03  2.76717557e-03  1.14503817e-03
  5.08905852e-04 -2.79898219e-03  3.16475827e-03  2.70356234e-04
  9.06488550e-04  1.55852417e-03 -1.35178117e-03  4.93002545e-04
 -1.46310433e-03 -1.03371501e-03  1.01781170e-03  9.70101781e-04
  1.46310433e-03 -1.57442748e-03 -1.01781170e-03  3.51304071e-02]


SpearmanrResult(correlation=0.7037136863307422, pvalue=3.5324927128e-313)